### Using prtorch to build a GAN zoo


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.utils.data

In [2]:
# load the data 
# using cifar10 data for DCGAN

dataset = datasets.CIFAR10(root='data/cifar10', download=True,
                           transform=transforms.Compose([
                               transforms.Scale(32),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])
                                      )

dataloader = torch.utils.data.DataLoader(dataset, batch_size=25,
                                         shuffle=True, num_workers=int(2))
len(dataloader)

Files already downloaded and verified


2000

* 数据加载
1. ToTensor是指把PIL.Image(RGB) 或者numpy.ndarray(H x W x C) 从0到255的值映射到0到1的范围内，并转化成Tensor格式。

### 按照DCGAN网络架构中的设计进行定义

* 首先需要定义的是BN和weight filler

In [3]:
def weight_filler(m):
    classname = m.__class__.__name__  #获取m的类型名
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [4]:
#使用一个类构建网络，
#通过super函数继承 nn.Module的构造方法
#使用sequential的方法构建网络模型

class G(nn.Module):
    def __init__(self):
        super(G, self).__init__() 
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 64 * 8, 4, 1, 0, bias=False), # 64*8 kernel nums; 4 kernel size; 1 stride; 0 padding
            nn.BatchNorm2d(64*8),
            nn.ReLU(True),
            # n kernel * 4 * 4
            nn.ConvTranspose2d(64*8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*4),
            nn.ReLU(True),
            # n kernel * 8 * 8
            nn.ConvTranspose2d(64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*2),
            nn.ReLU(True),
            # n kernel * 16 * 16
            nn.ConvTranspose2d(64 * 2, 3, 4, 2, 1, bias=False),
            nn.Tanh()
            # nc * 32 * 32
        ) 
        
    def forward(self, x):
        #x = x.view(x.size(0), x.size(1))
        return self.main(x)
g_model = G()

In [5]:
# 卷积操作的计算
# (W−F+2P)/S+1 : W: input size; F: kernel size; P:padding; S: stride;
class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
        nn.Conv2d(3, 64, 4, 2, 1, bias=False), # 64 kernel nums; 4 kernel size; 2 stride; 1 padding
        nn.LeakyReLU(0.2, inplace=True),
        # state size. (ndf) x 16 x 16
        nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
        nn.BatchNorm2d(64*2),
        nn.LeakyReLU(0.2, inplace=True),
         # state size. (ndf) x 8 x 8
        nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
        nn.BatchNorm2d(64*4),
        nn.LeakyReLU(0.2, inplace=True),
         # state size. (ndf) x 4 x 4
        nn.Conv2d(64 * 4, 1, 4, 1, 0, bias=False),
        nn.Sigmoid()
         # final output
        )
    def forward(self, x):
        output = self.main(x)
        return output
    
d_model = D()
print(d_model)

D (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid ()
  )
)


In [6]:
input_ = torch.FloatTensor(25, 3, 32, 32)
noise = torch.FloatTensor(25, 100, 1, 1) #(batch size; 100 dimension; 1 * 1)
fixed_noise = torch.FloatTensor(25, 100, 1, 1).normal_(0, 1)
label = torch.FloatTensor(25)

input_ = Variable(input_)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)
#noise.data.normal_(0,1)

In [7]:
#weight 初始化
g_model.apply(weight_filler)
d_model.apply(weight_filler)

# 设置求解器： DCGAN使用Adam进行求解； 学习率设置为0.0002
optimizerD = optim.Adam(d_model.parameters(), lr = 0.0002, betas = (0.5, 0.999)) 
optimizerG = optim.Adam(g_model.parameters(), lr = 0.0002, betas = (0.5, 0.999))

criterion = nn.BCELoss()

In [ ]:
for epoch in range(500):
    for i, data in enumerate(dataloader, 0):
        #update the D model with real data
        d_model.zero_grad()
        real, _ = data
        batch_size = real.size(0)
        input_.data.resize_(real.size()).copy_(real)
        label.data.resize_(batch_size).fill_(1) # real label is 1
       
        output = d_model(input_)

        loss_D_r = criterion(output, label)
        loss_D_r.backward()
        D_real = output.data.mean()
        #update D model with fake data
        noise.data.resize_(batch_size, 100, 1, 1)
        noise.data.normal_(0, 1)
        label.data.fill_(0) # fake label
        fake_input = g_model(noise)
        output = d_model(fake_input.detach())
        loss_D_f = criterion(output, label)
        loss_D_f.backward()
        D_fake = output.data.mean()
        
        errD = D_real + D_fake
        
        optimizerD.step()
        
        #update G mdoel
        g_model.zero_grad()
        label.data.fill_(1) # G model want the G samples be 1
        noise.data.resize_(batch_size, 100, 1, 1)
        noise.data.normal_(0, 1)
        #fake_input = g_model(noise)
        output = d_model(fake_input)
        
        loss_G = criterion(output, label)
        loss_G.backward()
        loss_D_G = output.data.mean()
        
        optimizerG.step()
        
        
        if i%100 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, 1000, i, len(dataloader),
                 D_real + D_fake, loss_D_G, D_real, D_fake, loss_D_G ))
            
            vutils.save_image(real, 
                             '%s/real_sample.png' % 'logs')
            fake = g_model(fixed_noise)
            vutils.save_image(fake.data,
                             '%s/fake_sample_epoch_%03d.png' % ('logs', epoch))
        
        

[0/1000][0/2000] Loss_D: 0.6828 Loss_G: 0.1366 D(x): 0.3119 D(G(z)): 0.3708 / 0.1366
[0/1000][100/2000] Loss_D: 0.9934 Loss_G: 0.0023 D(x): 0.9481 D(G(z)): 0.0453 / 0.0023
